## LLMRouterChain


The LLMRouterChain is a specialized chain in LangChain designed to route inputs dynamically to different chains or models based on some criteria. It is particularly useful in scenarios where you need to handle diverse tasks or inputs that require different processing methods.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

### Route Templates

In [2]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge of the student. Here is the question\n{input}'''

expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics. Here is the question\n{input}'''

# ADD OUR OWN TEMPLATES
empty_template = 'empty'

### Route Prompts

In [3]:
prompt_infos = [
    {'name':'empty', 'description':'Replies to empty questions', 'prompt_template':empty_template},
    {'name':'advanced physics', 'description':'Answers advanced physics questions','prompt_template':expert_template},
    {'name':'beginner physics', 'description':'Answers basic beginner physics questions', 'prompt_template':beginner_template},
]

### ConversationChain

In [4]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o")

In [5]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['prompt_template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    #chain = prompt | llm
    destination_chains[name] = chain

/var/folders/ny/r3wsv_bd0bs1_vcy4pb5q9pr0000gn/T/ipykernel_17448/2651825830.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [6]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=llm, prompt=default_prompt)

### Multi Routing Template

In [7]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

### Routing Destinations

In [8]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations = "\n".join(destinations)
print(destinations)

empty: Replies to empty questions
advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics questions


### Router Prompt

In [9]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
router_prompt = PromptTemplate(template=router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

In [10]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
empty: Replies to empty questions
advanced physics: Answers advanced physics questions
beginner physi

### Routing Chain Call

In [11]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [12]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True)

/var/folders/ny/r3wsv_bd0bs1_vcy4pb5q9pr0000gn/T/ipykernel_17448/2421999896.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains,


In [13]:
response = chain.invoke("How do magnets work?")
print(response['text'])



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.
Great question! Let's explore how magnets work in a simple way.

### What is a Magnet?

A magnet is a material or object that produces a magnetic field, which is an invisible force that can attract or repel certain materials, like iron, nickel, and cobalt.

### Basic Concepts

1. **Magnetic Poles**: Every magnet has two ends called poles - a North pole and a South pole. These poles are where the magnetic force is strongest. If you try to push the North pole of one magnet towards the North pole of another, they will repel each other. However, if you bring the North pole of one magnet close to the South pole of another, they will attract each other. This is often summarized by the phrase: "opposites attract, likes repel."

2. **Magnetic Field**: Around every magnet is an invisible area called the magnetic field. You can think of it as the area where the magnet's force is felt.

In [14]:
response = chain.invoke("Explain advanced concepts of Entropy in Thermodynamics")
print(response['text'])



> Entering new MultiPromptChain chain...
advanced physics: {'input': 'Explain advanced concepts of Entropy in Thermodynamics'}
> Finished chain.
Entropy is a central concept in thermodynamics and statistical mechanics, often embodying the idea of disorder or randomness in a system. In a more sophisticated sense, entropy is a measure of the number of microscopic configurations that correspond to a thermodynamic system's macroscopic state. Let's delve into some advanced aspects of entropy.

### Thermodynamic Entropy

In classical thermodynamics, entropy (\(S\)) is a state function that is central to the second law of thermodynamics. This law states that for any spontaneous process, the entropy of the universe increases. Mathematically, this can be expressed as:

\[ \Delta S_{\text{universe}} = \Delta S_{\text{system}} + \Delta S_{\text{surroundings}} \geq 0 \]

where \(\Delta S\) represents the change in entropy. For reversible processes, the equality holds, while for irreversible proc